Copied from [here](https://github.com/embl-cba/ome-zarr-image-analysis-nextflow/blob/main/playground/ngff-zarr/try-ngff-zarr.py) (thanks Christian)

In [2]:
import ngff_zarr

In [3]:
data_path = "../example-zarr"
file_path = f"{data_path}/13457539.zarr"

In [5]:
# open
input_multiscales = ngff_zarr.from_ngff_zarr(file_path)

In [6]:
# inspect one resolution
num_resolutions = len(input_multiscales.images)
image = input_multiscales.images[0]

data = image.data
scale = image.scale
translation = image.translation
units = image.axes_units

In [13]:
display("num_resolutions", num_resolutions)
display("image", image)
display("data", data)
display("scale", scale)
display("translation", translation)
display("units", units)

'num_resolutions'

2

'image'

NgffImage(data=dask.array<from-zarr, shape=(18, 6, 11, 189, 190), dtype=uint16, chunksize=(1, 1, 1, 189, 190), chunktype=numpy.ndarray>, dims=['t', 'c', 'z', 'y', 'x'], scale={'t': 1, 'c': 1, 'z': 1, 'y': 1, 'x': 1}, translation={'t': 0.0, 'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0}, name='image', axes_units={'t': None, 'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'})

'data'

...

{'t': 1, 'c': 1, 'z': 1, 'y': 1, 'x': 1}

'translation'

{'t': 0.0, 'c': 0.0, 'z': 0.0, 'y': 0.0, 'x': 0.0}

'units'

{'t': None, 'c': None, 'z': 'micrometer', 'y': 'micrometer', 'x': 'micrometer'}

In [10]:
# process one resolution
numpy_array = data.compute()
print("some pixel value:", numpy_array[0, 0, 0, 1, 1])
spatial_indices = [index for index, value in enumerate(image.dims) if value in ('x', 'y', 'z')]
numpy_array = numpy_array + 2
print("same pixel after adding 2:", numpy_array[0, 0, 0, 1, 1])

some pixel value: 0
same pixel after adding 2: 2


In [11]:
# save one resolution
output_image = ngff_zarr.to_ngff_image(numpy_array, image.dims, image.scale, image.translation, "processed", image.axes_units )

# Error: ModuleNotFoundError: No module named 'dask_image'
# https://github.com/thewtex/ngff-zarr/issues/53
output_multiscales = ngff_zarr.to_multiscales(output_image, 1)
ngff_zarr.to_ngff_zarr(f"{data_path}/output_ome_zarr", output_multiscales)

# the output can be opened with Napari, using the ome-zarr plugin